In [34]:
import requests
import psycopg2
import random
import folium
from folium.plugins import HeatMap
import openrouteservice

# Create synthetic Data

In [35]:
# Bereich für zufällige Breiten- und Längengrade in Berlin Kreuzberg/Friedrichshain
latitude_range = (52.49, 52.52)
longitude_range = (13.40, 13.45)

In [36]:
# Rating-Optionen
ratings = [1, 2, 3, 4, 5]

# Funktion zur Generierung zufälliger Koordinaten innerhalb des angegebenen Bereichs
def generate_random_coordinates():
    latitude = random.uniform(*latitude_range)
    longitude = random.uniform(*longitude_range)
    return latitude, longitude

# Funktion zur Generierung eines zufälligen Ratings
def generate_random_rating():
    return random.choice(ratings)

# OpenRouteService Client initialisieren
client = openrouteservice.Client(key='5b3ce3597851110001cf624842c9deb531d34e0795400d234fd30e06')  # Ersetzen Sie 'YOUR_API_KEY' durch Ihren API-Schlüssel


In [37]:
# Funktion zur Interpolation von Punkten zwischen zwei Koordinaten
def interpolate_points(coord1, coord2, num_points):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    interpolated_points = []
    for i in range(1, num_points + 1):
        lat = lat1 + (lat2 - lat1) * i / (num_points + 1)
        lon = lon1 + (lon2 - lon1) * i / (num_points + 1)
        interpolated_points.append((lat, lon))
    return interpolated_points

# Insert Routes

In [41]:
def insert_route(data, start_lat, start_lon, end_lat, end_lon, rating):
    # Connect to your PostgreSQL database
    conn = psycopg2.connect("dbname='bike_project' user='postgres' host='localhost' password='123456'")
    cur = conn.cursor()

    # Rating to weight adjustment mapping
    rating_weight_adjustment = {
        1: -2,
        2: -1,
        3: 0,
        4: 1,
        5: 2
    }

    # Insert route into the routes table with rating
    cur.execute("""
        INSERT INTO routes (start_latitude, start_longitude, end_latitude, end_longitude, rating) 
        VALUES (%s, %s, %s, %s, %s) RETURNING id
        """, (start_lat, start_lon, end_lat, end_lon, rating))
    route_id = cur.fetchone()[0]

    # Function to check if a node exists
    def get_node_id(latitude, longitude):
        cur.execute("SELECT id FROM nodes WHERE latitude = %s AND longitude = %s", (latitude, longitude))
        result = cur.fetchone()
        return result[0] if result else None

    # Insert nodes and route_nodes into the database
    sequence = 0
    coordinates = data['features'][0]['geometry']['coordinates']

    # List to store node IDs in the order of the route
    node_ids = []

    for coordinate in coordinates:
        longitude, latitude = coordinate
        node_id = get_node_id(latitude, longitude)
        
        if not node_id:
            cur.execute("INSERT INTO nodes (latitude, longitude) VALUES (%s, %s) RETURNING id", (latitude, longitude))
            node_id = cur.fetchone()[0]
        
        if (route_id, node_id) not in node_ids:
            cur.execute("INSERT INTO route_way (route_id, node_id, sequence) VALUES (%s, %s, %s)", (route_id, node_id, sequence))
            sequence += 1
            node_ids.append((route_id, node_id))

    # Function to check if an edge exists and return its weight
    def get_edge_weight(node_id_start, node_id_end):
        cur.execute("SELECT weight FROM edges WHERE node_id_start = %s AND node_id_end = %s", (node_id_start, node_id_end))
        result = cur.fetchone()
        return result[0] if result else None

    # Function to update edge weight
    def update_edge_weight(node_id_start, node_id_end, weight_adjustment):
        cur.execute("UPDATE edges SET weight = weight + %s WHERE node_id_start = %s AND node_id_end = %s", 
                    (weight_adjustment, node_id_start, node_id_end))

    # Insert edges into the database
    for i in range(len(node_ids) - 1):
        start_node_id = node_ids[i][1]
        end_node_id = node_ids[i + 1][1]
        weight_adjustment = rating_weight_adjustment[rating]
        
        current_weight = get_edge_weight(start_node_id, end_node_id)
        
        if current_weight is None:
            cur.execute("INSERT INTO edges (node_id_start, node_id_end, weight) VALUES (%s, %s, %s)", 
                        (start_node_id, end_node_id, weight_adjustment))
        else:
            update_edge_weight(start_node_id, end_node_id, weight_adjustment)

    # Commit the transaction
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()

# Call random Entries

In [44]:
num_entries = 200

for i in range(num_entries):
    # Generieren der zufälligen Start- und Endkoordinaten
    start_lat, start_lon = generate_random_coordinates()
    end_lat, end_lon = generate_random_coordinates()
    
    # Generieren des zufälligen Ratings
    rating = generate_random_rating()

    # Routeninformationen von OpenRouteService abrufen
    response = client.directions(coordinates=[[start_lon, start_lat], [end_lon, end_lat]], profile='driving-car', format='geojson')
    data = response

    insert_route(data, start_lat, start_lon, end_lat, end_lon, rating)

    print(data)

{'type': 'FeatureCollection', 'bbox': [13.424283, 52.504542, 13.441516, 52.51963], 'features': [{'bbox': [13.424283, 52.504542, 13.441516, 52.51963], 'type': 'Feature', 'properties': {'segments': [{'distance': 2390.5, 'duration': 377.7, 'steps': [{'distance': 1546.8, 'duration': 214.4, 'type': 11, 'instruction': 'Head northwest on Mühlenstraße', 'name': 'Mühlenstraße', 'way_points': [0, 32]}, {'distance': 600.9, 'duration': 111.6, 'type': 1, 'instruction': 'Turn right onto Lichtenberger Straße', 'name': 'Lichtenberger Straße', 'way_points': [32, 53]}, {'distance': 168.2, 'duration': 37.1, 'type': 7, 'instruction': 'Enter the roundabout and take the 2nd exit onto Strausberger Platz', 'name': 'Strausberger Platz', 'exit_number': 2, 'way_points': [53, 64]}, {'distance': 41.4, 'duration': 9.9, 'type': 1, 'instruction': 'Turn right onto Neue Weberstraße', 'name': 'Neue Weberstraße', 'way_points': [64, 66]}, {'distance': 33.3, 'duration': 4.8, 'type': 0, 'instruction': 'Turn left onto Lichte

c:\Users\bened\Desktop\Uni\Enterprise Architectures for Big Data\Project\env\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\bened\Desktop\Uni\Enterprise Architectures for Big Data\Project\env\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\bened\Desktop\Uni\Enterprise Architectures for Big Data\Project\env\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\bened\Desktop\Uni\Enterprise Architectures for Big Data\Project\env\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit 

{'type': 'FeatureCollection', 'bbox': [13.407219, 52.503406, 13.438738, 52.513372], 'features': [{'bbox': [13.407219, 52.503406, 13.438738, 52.513372], 'type': 'Feature', 'properties': {'segments': [{'distance': 3400.6, 'duration': 523.7, 'steps': [{'distance': 9.3, 'duration': 6.7, 'type': 11, 'instruction': 'Head northwest', 'name': '-', 'way_points': [0, 1]}, {'distance': 115.7, 'duration': 27.8, 'type': 1, 'instruction': 'Turn right onto Lobeckstraße', 'name': 'Lobeckstraße', 'way_points': [1, 7]}, {'distance': 178.9, 'duration': 23.1, 'type': 1, 'instruction': 'Turn right onto Oranienstraße', 'name': 'Oranienstraße', 'way_points': [7, 12]}, {'distance': 998.1, 'duration': 158.0, 'type': 7, 'instruction': 'Enter the roundabout and take the 3rd exit onto Prinzenstraße', 'name': 'Prinzenstraße', 'exit_number': 3, 'way_points': [12, 41]}, {'distance': 347.3, 'duration': 45.5, 'type': 1, 'instruction': 'Turn right onto Köpenicker Straße', 'name': 'Köpenicker Straße', 'way_points': [41,

# Map Creation

In [50]:
import psycopg2
import folium
import matplotlib.colors as mcolors
import matplotlib.cm as cm

# Funktion zur Bestimmung der Farbe basierend auf dem Gewicht
def get_color(weight, vmin, vmax):
    norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
    cmap = cm.RdYlGn  # Direkter Zugriff auf die Colormap
    return mcolors.to_hex(cmap(norm(weight)))

# Verbindung zur PostgreSQL-Datenbank
conn = psycopg2.connect("dbname='bike_project' user='postgres' host='localhost' password='123456'")
cur = conn.cursor()

# Abfrage aller Edges mit deren Start- und Endkoordinaten sowie Gewichten
cur.execute("""
    SELECT ns.latitude, ns.longitude, ne.latitude, ne.longitude, e.weight
    FROM edges e
    JOIN nodes ns ON e.node_id_start = ns.id
    JOIN nodes ne ON e.node_id_end = ne.id
""")
edges = cur.fetchall()

# Bestimmen des minimalen und maximalen Gewichts
weights = [edge[4] for edge in edges]
vmin = min(weights)
vmax = max(weights)

# Karte initialisieren
map_center = [52.5, 13.4]  # Beispielzentrum (Berlin)
mymap = folium.Map(location=map_center, zoom_start=12)

# Edges zur Karte hinzufügen
for edge in edges:
    start_lat, start_lon, end_lat, end_lon, weight = edge
    color = get_color(weight, vmin, vmax)
    folium.PolyLine(
        locations=[(start_lat, start_lon), (end_lat, end_lon)],
        color=color,
        weight=5,
        opacity=0.8
    ).add_to(mymap)

# HTML-Datei speichern
mymap.save('route_map.html')

# Verbindung schließen
cur.close()
conn.close()
